In [ ]:
import numpy as np
import time
from functools import partial

In [ ]:
def create_table():
    table = {}
    for a in range(16):
        for b in range(16):
            for c in range(16):
                for d in range(16):
                    score = 0
                    change = False
                    line = (a, b, c, d)
                    if (len(set(line)) == 4 and min(line)) or (not max(line)):
                        table[line] = (line, score, change)
                        continue
                    line_1 = [v for v in line if v]
                    for i in range(len(line_1) - 1):
                        x = line_1[i]
                        if x == line_1[i + 1]:
                            score += 1 << (x + 1)
                            change = True
                            line_1[i], line_1[i + 1] = x + 1, 0
                    line_2 = [v for v in line_1 if v]
                    line_2.extend([0] * (4 - len(line_2)))
                    if tuple(line_2) == line:
                        table[line] = (line, score, change)
                    else:
                        change = True
                        table[line] = (line_2, score, change)
    print('table of moves created')
    return table

  member of the class is the state of the 4*4 board,
  score = current score in the game
  odometer = number of moves from the start
  row = numpy array of shape (4, 4)
  numbers stored in the Game are 0 for 0 and log2(n) for 2,4,8 ..

In [ ]:
class Game:

    moves = {0: 'left', 1: 'up', 2: 'right', 3: 'down'}
    table = create_table()
    counter = 0

    def __init__(self, score=0, odometer=0, row=None):
        self.score = score
        self.odometer = odometer
        if row is None:
            self.history = []
            self.row = np.zeros((4, 4), dtype=np.int32)
            self.new_tile()
            self.new_tile()
        else:
            self.row = np.array(row, dtype=np.int32)
        self.history = []

    def copy(self):
        return Game(self.score, self.odometer, self.row)

    def __eq__(self, other):
        return np.array_equal(self.row, other.row)

    def __str__(self):
        return '\n'.join(['\t\t\t\t'.join([str(1 << val if val else 0) for val in j]) for j in self.row]
                         ) + '\n score = ' + str(self.score) + ' odometer = ' + str(self.odometer)

    def empty(self):
        return [(i, j) for j in range(4) for i in range(4) if self.row[i, j] == 0]

    def empty_count(self):
        return 16 - np.count_nonzero(self.row)

    def pair_count(self):
        state = self.row
        zero = np.count_nonzero(state[:, :3] - state[:, 1:]) + np.count_nonzero(state[:3, :] - state[1:, :])
        return 24 - zero

    def game_over(self):
        return not self.empty_count() and not self.pair_count()

    def new_tile(self, tile_position=None, inplace=True):
        if not tile_position:
            em = self.empty()
            if len(em) == 0:
                return False, (0, (0, 0))
            tile = 1 if np.random.randint(10) else 2
            position = em[np.random.randint(0, len(em))]
        else:
            tile, position = tile_position
        if inplace:
            self.row[position] = tile
            self.history.append((tile << 1, position))
        return tile, position

    # This could be a 8-10 line function. Compress zeros, merge, compress again,
    # and check if the move was valid by comparing arrays etc.
    # But that is significantly slower compared to the below brute-force enumeration of options.
    # The function gets executed gazillions of times during NN training, so speed is important.
    def _left(self):
        change = False
        for i in range(4):
            line, score, change_line = Game.table[tuple(self.row[i])]
            if change_line:
                change = True
                self.score += score
                self.row[i] = line
        return change

    def move(self, direction):
        Game.counter += 1
        self.history.append(self.copy())
        if direction:
            self.row = np.rot90(self.row, direction)
        change = self._left()
        if direction:
            self.row = np.rot90(self.row, 4 - direction)
        if change:
            self.odometer += 1
            self.history.append(direction)
        return change

    @staticmethod
    def trial_run(estimator, game_init=None, step_limit=100000, verbose=False):
        game = game_init or Game()
        while game.odometer < step_limit:
            if game.game_over():
                game.history.append(game)
                return game
            best_dir, best_value = 0, - np.inf
            for direction in range(4):
                variant = game.copy()
                change = variant.move(direction)
                if change:
                    value = estimator(variant)
                    if value > best_value:
                        best_dir, best_value = direction, value
            if verbose:
                print(game.odometer, ' ', Game.moves[best_dir])
                print(game)
            game.move(best_dir)
            game.new_tile()
        game.history.append(game)
        return game

    @staticmethod
    def trial(estimator, num=20, game=None, verbose=False):
        start = time.time()
        results = []
        figures = []
        for i in range(num):
            a = Game.trial_run(estimator, game_init=game, verbose=verbose)
            results.append(a)
            fig = 1 << np.max(a.row)
            figures.append(fig)
            print(f'game {i}, result {a.score}, achieved {fig}')
        average = np.average([a.score for a in results])
        results.sort(key=lambda b: b.score, reverse=True)

        def share(limit):
            return len([0 for i in figures if i >= limit]) / len(figures) * 100

        for a in results[:3]:
            print(a)
        elapsed = time.time() - start
        print(f'average score of {num} runs = {average}')
        print(f'4096 reached in {share(4096)}%')
        print(f'2048 reached in {share(2048)}%')
        print(f'1024 reached in {share(1024)}%')
        print(f'total time = {elapsed}')
        print(f'total number of moves = {Game.counter}')
        print(f'time per move = {elapsed / Game.counter * 1000} ms')
        return results

    def replay(self):
        for step in range(0, len(self.history) - 1, 3):
            state = self.history[step]
            move = Game.moves[self.history[step + 1]]
            new_tile, position = self.history[step + 2]
            print(state)
            print(f'next move = {move}')
            print(f'new tile = {new_tile} at position = {position}')
        print('no more moves possible, final position =')
        print(self.history[-1])

In [ ]:
def look_forward(game_init, depth=0, width=1, evaluator=None):
    empty = game_init.empty_count()
    num_tiles = min(width, empty)
    if empty > 3:
        num_tiles = 1
    average = 0
    for i in range(num_tiles):
        game_tile = game_init.copy()
        game_tile.new_tile()
        best_score = 0
        for direction in range(4):
            game = game_tile.copy()
            change = game.move(direction)
            if change:
                if depth:
                    score = look_forward(game, depth - 1, width, evaluator)
                else:
                    if evaluator:
                        score = evaluator(game)
                    else:
                        score = game.score
                best_score = max(score, best_score)
        average += best_score
    average /= num_tiles
    return average

In [ ]:
def estimator_lf(depth=0, width=1, evaluator=None):
    return partial(look_forward, depth=depth, width=width, evaluator=evaluator)

In [ ]:
def snake_eval(game):
    figures = np.sort(game.row, axis=None)[::-1]
    f_1, f_2 = figures[:4], figures[4:8]

    def _left_up_vertical(state):

        def _diff(line_1, line_2):
            return np.sum(1 << np.abs(line_1 - line_2))

        corner = 0 if f_1[0] == state[0, 0] else 1 << (f_1[0] + 1)
        diff_1 = _diff(state[0], f_1)
        diff_2_rev = _diff(state[1][::-1], f_2)
        return - (corner + diff_1 + diff_2_rev)

    best_value = - np.inf
    state = game.row.copy()
    for i in range(4):
        value = _left_up_vertical(state)
        best_value = max(value, best_value)
        state = np.transpose(state)
        value = _left_up_vertical(state)
        best_value = max(value, best_value)
        state = np.rot90(np.transpose(state))

    return max(game.score + best_value * 2, 1)

In [ ]:
def pairs_and_corners(game):
    flat = np.ravel(game.row)
    pos_f = np.argmax(flat)
    f = flat[pos_f]
    flat[pos_f] = 0
    pos_f2 = np.argmax(flat)
    f_2 = flat[pos_f2]
    flat[pos_f] = f
    is_corner = f if f in (game.row[0, 0], game.row[0, 3], game.row[3, 0], game.row[3, 3]) else 0
    is_middle = -f if f in (game.row[1, 1], game.row[1, 2], game.row[2, 1], game.row[2, 2]) else 0

    def near_cells(pos):
        adj = []
        if pos % 4 > 0:
            adj.append(pos - 1)
        if pos % 4 < 3:
            adj.append(pos + 1)
        if pos // 4 > 0:
            adj.append(pos - 4)
        if pos // 4 < 3:
            adj.append(pos + 4)
        return [flat[v] for v in adj]

    adj = near_cells(pos_f)
    is_next_2 = f_2 if f_2 in adj else -f_2
    if is_corner and is_next_2 > 0 and f != f_2:
        is_corner += 2
        is_next_2 += 2

    pairs = max(6, game.pair_count())
    result = 2 * np.log2(game.score + 1) + pairs + is_corner + is_middle + is_next_2
    return result

In [ ]:
def _mc(game, empty=3, score=5000, odo=6):
    if game.empty_count() > empty or game.score < score:
        return game.score
    a = game.copy()
    a.new_tile()
    b = Game.trial_run(estimator_lf(), game_init=a, step_limit=game.odometer + odo)
    return b.score

In [ ]:
def monte_carlo(empty=3, score=5000, odo=6):
    return partial(_mc, empty=empty, score=score, odo=odo)